## Prepare video files

In [5]:
import cv2
import numpy as np
from pathlib import Path
# from pathlib import PurePath 

In [22]:
def generate_resized_video(src_path, dest_dir):
    cap = cv2.VideoCapture(str(src_path))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    frac = 256 / 1080
    
    name = src_path.name
    output_file = dest_dir.joinpath(name)

    out = cv2.VideoWriter(str(output_file),fourcc, 5, (round(1920*frac), 256))
    while True:
        ret, frame = cap.read()
        if ret == True:
            b = cv2.resize(frame, (round(1920*frac), 256), fx=0, fy=0, interpolation = cv2.INTER_CUBIC)
            out.write(b)
        else:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [74]:
vcap = cv2.VideoCapture('output.avi')

if vcap.isOpened(): 
    width  = vcap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
    height = vcap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`
    print('width, height:', width, height)
    
    fps = vcap.get(cv2.CAP_PROP_FPS)    
    print('fps:', fps)  # float `fps`
    
    frame_count = vcap.get(cv2.CAP_PROP_FRAME_COUNT)    
    print('frames count:', frame_count)  # float `frame_count`
    
vcap.release()

width, height: 454.0 256.0
fps: 5.0
frames count: 204.0


### Resize video files in NTU60

In [ ]:
src_dir = Path("../../cvhci/data/activity/NTU_RGBD/rgb")
dest_dir = Path("./ntu60_rgb")
count = 0

for file in src_dir.iterdir():
    if file.suffix == '.avi':
        class_name = file.stem[17:20]
        if int(class_name) <= 60:
            generate_resized_video(file, dest_dir)
            count += 1 

In [6]:
from pathlib import Path
count = 0
for file in Path("./ntu60_rgb").iterdir():
    if file.suffix != '.csv' and file.stem[7] == '1':
        count+=1
print(count)

3887


### Generate csv files for test, val and train dataset

In [1]:
import csv

In [5]:
with open('./ntu60_rgb/test.csv', 'w', newline='') as file:
    writer = csv.writer(file)
#     field = ["path", "label"]
#     writer.writerow(field)
    
    for file in Path("./ntu60_rgb").iterdir():
        if file.suffix != '.csv' and file.stem != 'S007C001P007R001A049_rgb' and file.stem[7] == '1':
            label_id = int(file.stem[18:20]) - 1
            s = " ".join([str(file.name), str(label_id)])
            writer.writerow([s])

In [10]:
val_video = []

with open('./ntu60_rgb/val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
#     field = ["path", "label"]
#     writer.writerow(field)
    count = 0
    
    for file in Path("./ntu60_rgb").iterdir():
        if file.suffix != '.csv' and file.stem[7] != '1':
            s = " ".join([str(file.name), str(int(file.stem[18:20]) - 1)])
            writer.writerow([s])
            val_video.append(str(file.name))
            count += 1
        if count >= 1568:
            break

In [79]:
len(val_video)

1568

In [12]:
with open('./ntu60_rgb/train.csv', 'w', newline='') as file:
    count = 0
    writer = csv.writer(file)
#     field = ["path", "label"]
#     writer.writerow(field)
    
    for file in Path("./ntu60_rgb").iterdir():
        if file.suffix != '.csv' and file.stem[7] != '1' and str(file.name) not in val_video:
            s = " ".join([str(file.name), str(int(file.stem[18:20]) - 1)])
            writer.writerow([s])
            count +=1

In [81]:
count

6272

### Generate noisy label csv files of normal case based on csv files for video dataset

In [2]:
cd '~/mvitv2/noisy_label_20'

/home/prgc/mvitv2/noisy_label_20


In [3]:
rows = []
    
with open('./train.csv', newline='') as file:
    reader = csv.reader(file, delimiter=' ')
    for row in reader:
        s = " ".join([row[0], str(int(row[1]) + 1)])
        rows.append(s)

In [4]:
with open('../../ntu60_noise/noisy_train_20.csv', 'w', newline='') as file:        
    writer = csv.writer(file)
    for row in rows:
        writer.writerow([row])

In [5]:
with open('../../ntu60_noise/noisy_train_20.csv', newline='') as file:
    count = 0
    reader = csv.reader(file, delimiter=' ')
    for row in reader:
        label = int(row[0][18:20])
        if label != int(row[1]):
            count += 1
    print(count)

1252
